In [5]:
import numpy as np 
import pandas as pd

In [2]:
df_2018 = pd.read_csv('../01.웹/data_건강/건강검진정보_2018.csv', encoding='euc-kr')
df_2018.head(2)

,기준년도,가입자일련번호,성별코드,연령대코드(5세단위),시도코드,신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),...,감마지티피,흡연상태,음주여부,구강검진수검여부,치아우식증유무,결손치유무,치아마모증유무,제3대구치(사랑니)이상,치석,데이터공개일자
0,2018,1,2,7,48,160,60,79.5,1.5,1.5,...,12.0,1.0,NaN,N,NaN,NaN,NaN,NaN,NaN,2019-12-19
1,2018,2,1,6,26,170,55,69.3,1.2,0.8,...,47.0,1.0,Y,Y,0.0,NaN,NaN,NaN,N,2019-12-19


In [3]:
bmi_2018 =round(df_2018['체중(5Kg단위)'] /(df_2018['신장(5Cm단위)']/100) **2,1)
df_2018['BMI'] = bmi_2018
df_2018.head(3)

,기준년도,가입자일련번호,성별코드,연령대코드(5세단위),시도코드,신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),...,흡연상태,음주여부,구강검진수검여부,치아우식증유무,결손치유무,치아마모증유무,제3대구치(사랑니)이상,치석,데이터공개일자,BMI
0,2018,1,2,7,48,160,60,79.5,1.5,1.5,...,1.0,NaN,N,NaN,NaN,NaN,NaN,NaN,2019-12-19,23.4
1,2018,2,1,6,26,170,55,69.3,1.2,0.8,...,1.0,Y,Y,0.0,NaN,NaN,NaN,N,2019-12-19,19.0
2,2018,3,1,12,28,165,70,85.0,0.8,0.8,...,2.0,Y,Y,0.0,NaN,NaN,NaN,N,2019-12-19,25.7


In [4]:
male = df_2018['BMI'][df_2018.성별코드==1].value_counts().sum()
female = df_2018['BMI'][df_2018.성별코드 == 2].value_counts().sum()
male,female

(535600, 464400)

In [29]:
df_list = df_2018['BMI'].to_dict('records')
df_list


TypeError: unsupported type: <class 'str'>

In [27]:
ranges=[18.5,25,30,40,50]
bmi_bins=pd.cut(df_2018['BMI'],ranges).describe()
bmi_bins

count           955185
unique               4
top       (18.5, 25.0]
freq            607229
Name: BMI, dtype: object

In [16]:
df_2018['BMI'].min()


13.3

In [6]:
def bmi_cal(x, y):       # x이상 y미만
    if y == 18.5:
        return df_2018[(df_2018['BMI'] < y)].groupby(['성별코드','연령대코드(5세단위)'])['BMI'].count()
    elif x != 30 and y != 18.5:
        return df_2018[(df_2018['BMI'] < y) & (df_2018['BMI'] >= x)].groupby(['성별코드','연령대코드(5세단위)'])['BMI'].count()
    elif x == 30:
        return df_2018[(df_2018['BMI'] >= x)].groupby(['성별코드','연령대코드(5세단위)'])['BMI'].count()

temp = pd.DataFrame()
for x, y in zip(np.arange(13, 30, 5.5), np.arange(18.5, 45 ,5.5)):
    temp = pd.concat([temp, bmi_cal(x, y).to_frame()], axis=1)
temp.columns = ['18.5','18.5-25.0','25.0-30.0','30']

In [7]:
temp.iloc[1, 0] = 0
temp=temp.T
temp

성별코드              1                                                        \
연령대코드(5세단위)      5        6        7        8        9        10       11   
18.5          582.0      0.0    835.0    944.0   1093.0   1060.0   1265.0   
18.5-25.0    4864.0  15350.0  18136.0  20460.0  23850.0  23255.0  26386.0   
25.0-30.0    3577.0  16150.0  25682.0  33325.0  37394.0  34769.0  35736.0   
30            515.0   2552.0   4285.0   4942.0   4816.0   3429.0   2765.0   

성별코드                                    ...        2                    \
연령대코드(5세단위)       12       13       14  ...       9        10       11   
18.5          1267.0   1252.0    824.0  ...   4971.0   3042.0   2594.0   
18.5-25.0    24366.0  23786.0  13856.0  ...  34585.0  32597.0  38219.0   
25.0-30.0    28913.0  25806.0  13497.0  ...  12891.0  14904.0  21794.0   
30            1700.0   1411.0    600.0  ...   1918.0   1762.0   2262.0   

성별코드                                                                     
연령대코드(5세단위)       12       13       14       15      16      17      18  
18.5          1805.0   1404.0    620.0    681.0   515.0   549.0   324.0  
18.5-25.0    30773.0  27829.0  13670.0  12542.0  7374.0  5057.0  1522.0  
25.0-30.0    20380.0  23210.0  13745.0  13094.0  6852.0  3756.0   682.0  
30            1930.0   2419.0   1417.0   1419.0   750.0   456.0    28.0  

[4 rows x 28 columns]

In [33]:
import pandas as pd
import numpy as np

def bmi_cal(x, y):
    if y == 18.5:
        return df_2018[(df_2018['BMI'] < y)].groupby('연령대코드(5세단위)')['BMI'].count()
    elif x != 30 and y != 18.5:
        return df_2018[(df_2018['BMI'] < y) & (df_2018['BMI'] >= x)].groupby('연령대코드(5세단위)')['BMI'].count()
    elif x == 30:
        return df_2018[(df_2018['BMI'] >= x)].groupby('연령대코드(5세단위)')['BMI'].count()

temp = pd.DataFrame()
for x, y in zip(np.arange(13, 30, 5.5), np.arange(18.5, 45, 5.5)):
    temp = pd.concat([temp, bmi_cal(x, y).to_frame()], axis=1)

temp.columns = ['18.5', '18.5-25.0', '25.0-30.0', '30']
temp.iloc[1, 0] = 0

total_counts = temp.sum(axis=1)
temp['비율'] = temp.sum(axis=1) / total_counts.sum() * 100
temp = temp.round(2)

print(temp)


             18.5  18.5-25.0  25.0-30.0    30     비율
연령대코드(5세단위)                                         
5            2655      12334       5441   787   2.15
6               0      34461      20411  3227   5.88
7            5233      35868      30320  5092   7.75
8            4309      37528      38536  5769   8.72
9            6064      58435      50285  6734  12.30
10           4102      55852      49673  5191  11.62
11           3859      64605      57530  5027  13.26
12           3072      55139      49293  3630  11.25
13           2656      51615      49016  3830  10.84
14           1444      27526      27242  2017   5.90
15           1497      25176      24422  1874   5.36
16           1071      14334      11909   893   2.86
17            988       9273       6195   507   1.72
18            405       2379        984    28   0.38


In [36]:
temp['비율'].values

array([ 2.15,  5.88,  7.75,  8.72, 12.3 , 11.62, 13.26, 11.25, 10.84,
        5.9 ,  5.36,  2.86,  1.72,  0.38])

In [39]:
import pandas as pd
import numpy as np

def bmi_cal(x, y):
    if y == 18.5:
        return df_2018[(df_2018['BMI'] < y)]['BMI'].count()
    elif x != 30 and y != 18.5:
        return df_2018[(df_2018['BMI'] < y) & (df_2018['BMI'] >= x)]['BMI'].count()
    elif x == 30:
        return df_2018[(df_2018['BMI'] >= x)]['BMI'].count()

temp = pd.DataFrame()
for x, y in zip(np.arange(13, 30, 5.5), np.arange(18.5, 45, 5.5)):
    temp = temp.append({'범위': f'{x}-{y}', '개수': bmi_cal(x, y)}, ignore_index=True)

total_count = temp['개수'].sum()
temp['비율'] = (temp['개수'] / total_count) * 100
temp = temp.round(2)

print(temp)


C:\Users\Admin\AppData\Local\Temp\ipykernel_26496\211038289.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append({'범위': f'{x}-{y}', '개수': bmi_cal(x, y)}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_26496\211038289.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append({'범위': f'{x}-{y}', '개수': bmi_cal(x, y)}, ignore_index=True)


          범위      개수     비율
0  13.0-18.5   43944   4.42
1  18.5-24.0  484525  48.73
2  24.0-29.5  421257  42.37
3  29.5-35.0   44606   4.49


C:\Users\Admin\AppData\Local\Temp\ipykernel_26496\211038289.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append({'범위': f'{x}-{y}', '개수': bmi_cal(x, y)}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_26496\211038289.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append({'범위': f'{x}-{y}', '개수': bmi_cal(x, y)}, ignore_index=True)


In [40]:
male=temp['비율'].value_counts()
male.index


Float64Index([4.42, 48.73, 42.37, 4.49], dtype='float64')

In [42]:
# index값 넣기
male_age = [18.5, 18.5-24.0, 24.0-29.5, 29.5-35.0]
male_pct = [4.42, 48.73, 42.37, 4.49]
temp_male = [] 
for m, n in zip(male_age, male_pct):
    temp_male.append({'BMI': m, '비율': n})
temp_male

[{'BMI': -5.5, '비율': 4.42},
 {'BMI': -5.5, '비율': 48.73},
 {'BMI': -5.5, '비율': 42.37},
 {'BMI': -5.5, '비율': 4.49}]

In [1]:
def calculate_bmi(weight, height):
    bmi = round(weight /(height/100) **2,1) # 체질량지수 계산
    age_average_bmi = 22  # 연령대 평균 체질량지수
    percentile = 31  # 체질량지수 백분위

    bmi_category = ""
    if bmi < 18.5:
        bmi_category = "저체중"
    elif bmi < 23:
        bmi_category = "정상"
    elif bmi < 25:
        bmi_category = "과체중"
    else:
        bmi_category = "비만"

    result = f"당신의 체질량지수는 ({bmi}) 입니다.\n"
    result += f"연령대 평균 체질량지수는 ({age_average_bmi}) 입니다.\n"
    result += f"당신의 체질량지수는 연령대 상위 ({percentile}%) 입니다.\n"
    result += f"당신의 체질량지수는 ({bmi_category}) 입니다."

    return result

weight = float(input("몸무게(kg)를 입력하세요: "))
height = float(input("키(m)를 입력하세요: "))

result = calculate_bmi(weight, height)
print(result)


당신의 체질량지수는 (23.4) 입니다.
연령대 평균 체질량지수는 (22) 입니다.
당신의 체질량지수는 연령대 상위 (31%) 입니다.
당신의 체질량지수는 (과체중) 입니다.


In [3]:
def calculate_bmi(weight, height):
    bmi = round(weight /(height/100) **2,1) # 체질량지수 계산

    bmi_category = ""
    if bmi < 18.5:
        bmi_category = "저체중"
    elif bmi < 23:
        bmi_category = "정상"
    elif bmi < 25:
        bmi_category = "과체중"
    else:
        bmi_category = "비만"

    result = f"당신의 체질량지수는 ({bmi}) 입니다.\n"
    result += f"당신의 체질량지수는 ({bmi_category}) 입니다."


    return result

# weight = float(input("몸무게(kg)를 입력하세요: "))
# height = float(input("키(m)를 입력하세요: "))

# result = calculate_bmi(weight, height)
# print(result)

당신의 체질량지수는 (24.2) 입니다.
당신의 체질량지수는 (과체중) 입니다.


- 평균 체질량 구하기

In [19]:
bmi_2018 =round(df_2018['체중(5Kg단위)'] /(df_2018['신장(5Cm단위)']/100) **2,1)
df_2018['BMI'] = bmi_2018
df_2018.head(3)

,기준년도,가입자일련번호,성별코드,연령대코드(5세단위),시도코드,신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),...,흡연상태,음주여부,구강검진수검여부,치아우식증유무,결손치유무,치아마모증유무,제3대구치(사랑니)이상,치석,데이터공개일자,BMI
0,2018,1,2,7,48,160,60,79.5,1.5,1.5,...,1.0,NaN,N,NaN,NaN,NaN,NaN,NaN,2019-12-19,23.4
1,2018,2,1,6,26,170,55,69.3,1.2,0.8,...,1.0,Y,Y,0.0,NaN,NaN,NaN,N,2019-12-19,19.0
2,2018,3,1,12,28,165,70,85.0,0.8,0.8,...,2.0,Y,Y,0.0,NaN,NaN,NaN,N,2019-12-19,25.7


In [20]:
df_BMI_2018=df_2018.groupby(['연령대코드(5세단위)','성별코드'])['BMI'].sum().round(1)
df_BMI_2018=pd.DataFrame(df_BMI_2018)
df_BMI_2018.head(3)

BMI
연령대코드(5세단위) 성별코드          
5           1     228693.2
            2     252330.9
6           1     877504.1

In [21]:
# 각연령대 인원
df_age_2018=df_2018.groupby(['연령대코드(5세단위)','성별코드'])['연령대코드(5세단위)'].count().round(1)
df_age_2018=pd.DataFrame(df_age_2018)
df_age_2018.head(2)

연령대코드(5세단위)
연령대코드(5세단위) 성별코드             
5           1            9619
            2           11732

In [22]:
df_last = pd.concat([df_BMI_2018,df_age_2018], axis=1)
df_last.head(2)

BMI  연령대코드(5세단위)
연령대코드(5세단위) 성별코드                       
5           1     228693.2         9619
            2     252330.9        11732

In [23]:
last=(df_last['BMI']/df_last['연령대코드(5세단위)']).round(2)
df_last['평균 체질량'] =last
df_last.head(3)

BMI  연령대코드(5세단위)  평균 체질량
연령대코드(5세단위) 성별코드                               
5           1     228693.2         9619   23.78
            2     252330.9        11732   21.51
6           1     877504.1        35523   24.70

In [34]:
filtered_data = df_last[df_last['성별코드'] == 2]['평균 체질량'].values
list =[]
for height in filtered_data:
    list.append(height)
print(list)


[21.51, 21.37, 21.8, 22.17, 22.61, 23.0, 23.39, 23.59, 24.01, 24.33, 24.39, 24.15, 23.7, 22.2]


In [35]:

male_age = [ '20~24세', '25~29세', '30~34세', '35~39세', '40~44세', '45~49세', '50~54세', '55~59세',
'60~64세', '65~69세', '70~74세', '75~79세','80~84세', '85세+']
male_pct = [21.51, 21.37, 21.8, 22.17, 22.61, 23.0, 23.39, 23.59, 24.01, 24.33, 24.39, 24.15, 23.7, 22.2]
temp_male = []
for m, n in zip(male_age, male_pct):
    temp_male.append({'나이': m, '평균체질량': n})
temp_male

[{'나이': '20~24세', '평균체질량': 21.51},
 {'나이': '25~29세', '평균체질량': 21.37},
 {'나이': '30~34세', '평균체질량': 21.8},
 {'나이': '35~39세', '평균체질량': 22.17},
 {'나이': '40~44세', '평균체질량': 22.61},
 {'나이': '45~49세', '평균체질량': 23.0},
 {'나이': '50~54세', '평균체질량': 23.39},
 {'나이': '55~59세', '평균체질량': 23.59},
 {'나이': '60~64세', '평균체질량': 24.01},
 {'나이': '65~69세', '평균체질량': 24.33},
 {'나이': '70~74세', '평균체질량': 24.39},
 {'나이': '75~79세', '평균체질량': 24.15},
 {'나이': '80~84세', '평균체질량': 23.7},
 {'나이': '85세+', '평균체질량': 22.2}]